### Here is the implementation of YOLOv3 loss function. It is differnet from v1. see https://towardsdatascience.com/yolo-v3-object-detection-53fb7d3bfe6b

#### Before this, let's recap what have we done.  Dataloader reads image data and annotations. Darknet53, this net extracts features. Yolo_layers do the detections.
#### The yolo layers return 3 scale outputs: (13\*13 + 26\*26 + 52\*52)\*3\*(80+4+1). Here we using outputs and annotations to calculate the loss.

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def build_target(targets, pred):
    # targets: torch.Size([116, 6]), from the dataloader, 116 means num of objs readed from batch iamges. index + 类别 + 坐标 = 6, see 04_DataLoader.ipynb
    # pred:  pred: [[1,3,13,13,85],[1,3,26,26,85],[1,3,52,52,85]] from the net output
    

In [ ]:
def Yolov3Loss(nn.Module):
    def __init__(self, anchors, img_dim, numClass):
        super().__init__()
        

In [17]:
x = torch.tensor(2.,requires_grad = True)

In [19]:
x

tensor(2., requires_grad=True)

In [20]:
def loss(x):
    return x**2

In [21]:
l = loss(x)

In [22]:
l.backward()

In [23]:
l

tensor(4., grad_fn=<PowBackward0>)